In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
df = pd.read_csv('../raw_data/df_limpio_con_df_lunes_limpio.csv')
df.head()

In [ ]:
X = data.Tweet
y = data.label

X_train, X_test, y_train, y_test = train_test_split(X , y ,test_size = 0.3 )

In [ ]:
import nltk
nltk.download('punkt')

X_train = [word_tokenize(str(_)) for _ in X_train]
X_test = [word_tokenize(str(_)) for _ in X_test]

In [ ]:
import gensim.downloader as api

info = api.info()  # show info about available models/datasets
model = api.load("glove-twitter-25")  # download the model and return as object ready for use
model.most_similar("depression")

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
tk = Tokenizer()
tk.fit_on_texts(X)
vocab_size = len(tk.word_index)
vocab_size

In [ ]:
def embed_sentence(model, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in model:
            embedded_sentence.append(model[word])
        
    return np.array(embedded_sentence)
def embedding(model, sentences):
    
    embed = []
    
    for sentence in sentences:
        embedded_sentence = embed_sentence(model, sentence)
        embed.append(embedded_sentence)
        
    return embed
    
X_train = embedding(model, X_train)
X_test = embedding(model, X_test)

In [ ]:
X_train_pad = pad_sequences(X_train, dtype='float', padding='post', maxlen= 400,truncating= 'post')
X_test_pad = pad_sequences(X_test, dtype='float', padding='post', maxlen = 400 , truncating = 'post')

In [ ]:
from tensorflow.keras import layers, Sequential

embedding_size = 100

model = Sequential()
model.add(layers.Embedding(input_dim=vocab_size+1, 
                          input_length=140, 
                          output_dim=embedding_size,
                          mask_zero=True))
model.add(layers.LSTM(50, return_sequences=True))
model.add(layers.Dense(100, activation="relu"))
model.add(layers.Dense(1, activation="sigmoid"))

In [ ]:
model.summary()


In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy', 'AUC','Precision','Recall'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience=3, restore_best_weights=True)

history_fit = model.fit(X_train_pad, y_train, 
          batch_size = 32,
          epochs=100,
          validation_split=0.3,
          callbacks=[es],
          verbose = 1
         )

In [ ]:
history_fit.score

In [ ]:
# import torch
# import transformers as ppb
# from transformers import BertTokenizer

# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased',
#                                          do_lower_case = True)
# encoded_data_train = tokenizer.batch_encode_plus(X_train.text.values,
#                                                 add_special_tokes = True,
#                                                 return_attention_mask = True,
#                                                 pad_to_max_length = True,
#                                                 max_length = 256,
#                                                 return_tensors = 'pt')
# input_ids_train = encoded_data_train['input_ids']
# attention_masks_train = encoded_data_train['attention_mask']
# labels_train = torch.tensor(X_train.label.values)

In [ ]:
# from transformers import BertForSequenceClassification

# model = BertForSequenceClassification.from_pretrained('bert-base-uncased',
#                                                       num_labels = [0 , 1],
#                                                       output_attentions = False,
#                                                       output_hidden_states = False)